In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

cwd = '/content/drive/MyDrive/2023_이어드림/모의3차/'

In [4]:
!pip install transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.1 MB/s eta 0:00:00


In [5]:
!pip install "sacrebleu[ko]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.2/33.2 MB 30.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mecab-ko-dic: filename=mecab_ko_dic-1.0.0-py3-none-any.whl size=33424393 sha256=ef6b4976c93a50d8265fcdc131c0660eca689d5bf68184c34ca323da6cf71c7f
  Stored in directory: /root/.cache/pip/wheels/0d/46/b8/996432c9998690a5a66fb82f6499222e48a23ff5b6c7d80322
Successfully built mecab-ko-dic


In [6]:
import pandas as pd

train_df = pd.read_csv(os.path.join(cwd,'train.csv'))
train_df.head()

,id,text,mt,target
0,FAQEQ1Rt1ncRSxO,The winner of the first season of the show Gol...,쇼의 첫 번째 시즌의 우승자인 Golos Dina Garipova는 여배우 Irin...,쇼의 첫 번째 시즌의 우승자인 골로스디나 가리포바는 여배우 이리나 무라베바에게 헌정...
1,JSjY2hYQT4oPdz6,"Replica census entry for Gypsie Runningcloud, ...",Gypsie Runningcloud에 대한 복제 인구 조사 항목(48세).,"집시 런닝클라우드의 인구조사 자료 사본, 48세."
2,nykRvvrs3DjlWTQ,I don't think I've ever seen one where they in...,나는 그들이 대화를 포함하는 것을 본 적이 없다고 생각합니다.,저는 그것들이 대화를 포함하는 것을 본 적이 없다고 생각합니다.
3,yIzKFCfNBhLx2q5,The patients were not significantly slower on ...,"환자들은 회상, 반응 시간 또는 추론 테스트에서 유의하게 느려지지 않았습니다.","환자들은 회상, 반응 시간 또는 추론 테스트에서 유의하게 느려지지 않았습니다."
4,68Yvljqe5BS5CWQ,"The artist declares, despite her outward fragi...",그 예술가는 겉으로는 연약함에도 불구하고 엄격한 지도자라고 선언한다.,그 예술가는 겉으로는 연약함에도 불구하고 엄격한 지도자라고 선언한다.


In [7]:
from transformers import pipeline

In [29]:
pipe = pipeline(task='text2text-generation', model='facebook/m2m100_418M', device='cuda')

In [15]:
pred = pipe(list(train_df.loc[:5,'text']), forced_bos_token_id=pipe.tokenizer.get_lang_id(lang='ko'))

print('original text:', train_df.loc[0,'text'])
print('target:',train_df.loc[0,'target'])
print('prediction:',pred[0]['generated_text'])

from sacrebleu.metrics import BLEU
bleu = BLEU(trg_lang='ko')

print(bleu.corpus_score([r['generated_text'] for r in pred], [list(train_df.loc[:5,'target'])]).score)

original text: The winner of the first season of the show GolosDina Garipova sings a song from the film Carnival in a program dedicated to actress Irina Murav'eva.
target: 쇼의 첫 번째 시즌의 우승자인 골로스디나 가리포바는 여배우 이리나 무라베바에게 헌정된 프로그램에서 영화 카니발의 노래를 부릅니다.
prediction: 쇼의 첫 번째 시즌의 우승자 GolosDina Garipova는 영화 카니발에서 여배우 Irina Murav'eva에 전념하는 프로그램에서 노래를 부릅니다.
26.35921813968538


In [36]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [37]:
from torch.utils.data import Dataset
from tqdm import tqdm

test_df = pd.read_csv(os.path.join(cwd,'test.csv'))
submission_df = pd.read_csv(os.path.join(cwd,'sample_submission.csv'))

class TestDataset(Dataset):
  def __init__(self, text):
    self.lst = list(text)

  def __len__(self):
    return len(self.lst)

  def __getitem__(self,i):
    return self.lst[i]

test_dataset = TestDataset(test_df['text'])

preds = [i for i in tqdm(pipe(test_dataset,
                        forced_bos_token_id=pipe.tokenizer.get_lang_id(lang='ko')))]

  1%|▏         | 135/10757 [01:58<2:35:16,  1.14it/s]


KeyboardInterrupt: ignored

In [44]:
test_df = pd.read_csv(os.path.join(cwd,'test.csv'))
submission_df = pd.read_csv(os.path.join(cwd,'sample_submission.csv')).iloc[:100,:]

preds = pipe(list(test_df.loc[:99,'text']),
             batch_size = 16,
             forced_bos_token_id=pipe.tokenizer.get_lang_id(lang='ko'))
# 100개 약 20초

In [45]:
submission_df['pred'] = [r['generated_text'] for r in preds]
submission_df.tail()

,id,pred
95,bBZCAzHSOOGFqZa,"그녀는 항상 그가 어디에 있는지 알고 있으며, 이 여성들 중 일부는 여러 남성이 붙..."
96,CXUcqiixDb7FMfp,백신 접종하기로 결정한 소년은 의회에서 그의 어머니의 비 백신 아이디어를 파괴합니다.
97,W85K6AxoHPNqvPj,승무원 Aksel E. Ørstavik와 나는 노르웨이의 캡틴과 해양 생물학자 An...
98,PqpR0tRNaajS6pO,근처에 Rose Bowl 이 장소는 Factory Tea Bar라고합니다.
99,PAr1UI6h1OGnzeu,Shaw는이 약물이 백신 접종되지 않은 사람들을위한 백신을 대체하지 않는다고 지적합니다.


In [ ]:
submission_df.to_csv(os.path.join(cwd,'prediction.csv'), index=False)